Copyright Scott Jensen, San Jose State University

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">This ds4all notebook</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Scott Jensen,Ph.D.</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

# Basic Wrangling

In this notebook we are going to introduce some basic ideas about PySpark (since we will be using Python).  
The documentation for the classes and functions in that module can be found <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html" target="_blank">here</a>.
If you click on that link, it can be a bit intimidating - remember that page is documenting *every* method and function in PySpark.  At the top of that
page are the major classes in the PySpark SQL module.  We won't be using most of the methods and functions, but in this exercise and future notebooks and examples we will walk through some you will be using in your project.

We will start with the SparkSession, which is created by Databricks when you spun up your cluster and it's always named `spark` by convention (not just in Databricks - that's a Spark convention).

## Start With A Question: What metro areas are in the yelp dataset?  

The Yelp Dataset Challenge tells us there is data for 10 metro areas in the dataset, but it does not tell us 
which metro areas.  This means we need to do some data wrangling to find out.  We need to ask the data 
which metro areas are included in the data.  Is this set based or individual value based profiling?

First, we need to read our data. From the metadata provided by Yelp, we know the business data includes 
information on where each business is located: latitude and longitude, postal code, state, city, neighborhood, address so we
will start by reading the Yelp business data into a DataFrame.  

We will walk through reading our JSON files (zipped up in the bzip2 format). 
In future exercises we will just say we are reading the data - from experience you will know how to do that (or know which
of your notebooks to look in for an example).

To read the data file, we start with the `read` method from our `SparkSession` variable named `spark`.

If you look at the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession.read" target="_blank">PySpark documentation for the `read` method</a>, you will see that it returns a `DataFrameReader` 
which "can be used to read data in as a DataFrame".  If we entered the following in a cell, the variable we are creating on the left side of the = sign (named `dfr`) would be populated with an instance of a `DataFrameReader`.

`dfr = spark.read`

However, we want to read the business data into a DataFrame, not create a DataFrameReader, so we won't stop there.  You will see 
a <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader" target="_blank">link</a> for the 
documentation for a `DataFrameReader`, click on it.  The first method you will probably see for 
a `DataFrameReader` is the `csv()` method which reads data in from a csv file such as our data from the 
Social Security Administration.  However, we have a JSON file, 
so <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json" target="_blank">scroll on down to the `json` method</a>.
You will see there are a lot of possible options, but for our purposes the defaults are fine, we just need to provide a path.  
In the cell below, add the path to your business data as a string (meaning the path is enclosed in quotes).  If you are not sure what that
path is, go take a look at your DBFS and come back here.

The result of the `json` method of the `DataFrameReader` is a DataFrame, so we assign the result to a variable named `df_business`.
Keep in mind what is happening here.  The code `spark.read` generates a `DataFrameReader`, so the "dot" after that code is saying 
to call the `json` method from that class on that `DataFrameReader` which was created.

## Step 1: Loading the data

The first line reads your Yelp business data file.  Be sure it has the correct path to your data.

The last two lines of the cell print out a count of the number of businesses in the data file and the schema inferred by Spark for the data file.

In [0]:
df_business = spark.read.json('/yelp/business.bz2')
print ("record count:", df_business.count() )
df_business.printSchema()

record count: 209393
root
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

md ## Step 2: What does our data look like?

The DataFrame class has a method named `show()` that will show us a number of rows / records in our data; 20 rows is the default.  Since some fields, such as the business address or the text of a review can be long, the default is to truncate how many characters is shown for each column and only show the first 20 characters (truncate is another way of saying "chop off").  We can include optional parameters to say how many rows we want and whether to truncate the fields (or how many characters to truncate at).

In the following cell we are going to show the following fields from the Yelp business data:
* business_id
* name
* city
* state
* review_count
* stars
* hours (when they are open)
* RestaurantsPriceRange2 (within attributes, this is a value 1 - 4 to indicate how expensive it is)

The DataFrame method we will use is named `select` which is documented <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select" target="_blank">here</a>.

The `select` method takes the names of fields in a DataFrame as its parameters (as many as you want back), and returns a DataFrame with those columns.  Since we want to just
see the data, we will call the `show` method directly on the DataFrame generated by `select`, we just need to include the dot in between to say "pass our new DataFrame to the `show` method".

In [0]:
df_busSelected=df_business.select("business_id", "name", "city", "state", "review_count", "stars", "hours", "attributes.RestaurantsPriceRange2")
df_busSelected.show(10,truncate=22)

+----------------------+----------------------+---------------+-----+------------+-----+----------------------+----------------------+
 business_id| name| city|state|review_count|stars| hours|RestaurantsPriceRange2|
+----------------------+----------------------+---------------+-----+------------+-----+----------------------+----------------------+
f9NumwFMBDn751xgFiRbNA|The Range At Lake N...| Cornelius| NC| 36| 3.5|[11:0-20:0, 10:0-18...| 3|
Yzvjg0SayhoZgCljUJRF9Q| Carlos Santo, NMD| Scottsdale| AZ| 4| 5.0| null| null|
XNoUzKckATkOD1hP6vghZg| Felinus| Montreal| QC| 5| 5.0| null| null|
6OAZjbxqM5ol29BuHsil3w| Nevada House of Hose|North Las Vegas| NV| 3| 2.5|[7:0-16:0, 7:0-16:0...| 4|
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES...| Mesa| AZ| 26| 4.5|[9:0-16:0, 0:0-0:0,...| null|
cKyLV5oWZJ2NudWgqs8VZw|Oasis Auto Center -...| Gilbert| AZ| 38| 4.5|[7:0-18:0, 7:0-18:0...| null|
oiAlXZPIFm2nBCt0DHLu_Q| Green World Cleaners| Las Vegas| NV| 81| 3.5|[7:0-19:0, 7:0-19:0...| 1|
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Aut...| Mesa| AZ| 18| 5.0|[7:30-17:0, 7:30-17...| null|
pQeaRpvuhoEqudo3uymHIQ| The Empanadas House| Champaign| IL| 5| 4.5|[11:30-14:30, 11:30...| 1|
EosRKXIGeSWFYWwpkbhNnA| Xtreme Couture| Toronto| ON| 16| 3.0|[5:30-23:0, 5:30-23...| null|
+----------------------+----------------------+---------------+-----+------------+-----+----------------------+----------------------+
only showing top 10 rows

In [0]:
type(df_busSelected)

Out[9]: pyspark.sql.dataframe.DataFrame

In [0]:
df_business.show(2,truncate=False,vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------
 address | 10913 Bailey Rd 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, False,,,,,,,, False,,,,,,,,,,,,, 3,,,,,,] 
 business_id | f9NumwFMBDn751xgFiRbNA 
 categories | Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping 
 city | Cornelius 
 hours | [11:0-20:0, 10:0-18:0, 11:0-20:0, 13:0-18:0, 11:0-20:0, 11:0-20:0, 10:0-18:0] 
 is_open | 1 
 latitude | 35.4627242 
 longitude | -80.8526119 
 name | The Range At Lake Norman 
 postal_code | 28031 
 review_count | 36 
 stars | 3.5 
 state | NC 
-RECORD 1------------------------------------------------------------------------------------------------------------------------------------------------------------
 address | 8880 E Via Linda, Ste 107 
 attributes | [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,] 
 business_id | Yzvjg0SayhoZgCljUJRF9Q 
 categories | Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates 
 city | Scottsdale 
 hours | null 
 is_open | 1 
 latitude | 33.5694041 
 longitude | -111.8902637 
 name | Carlos Santo, NMD 
 postal_code | 85258 
 review_count | 4 
 stars | 5.0 
 state | AZ 
only showing top 2 rows

## Step 3: Profiling for the metro areas

The closest field in the data to the concept of a metro area is a city, so let's see how many cities we have in the data.

For the above cell, name the resulting DataFrame `df_busSelected`

Using our new DataFrame, we will select the `city` and `state` fields.

We really only want each city and state combination listed once, so Emerald Chinese Restaurant is in Mississauga Ontario (ON), but we only want that city listed once, not once for each business.  If there is a Mississauga Arizona, we would also want that listed.
To accomplish this, there is a DataFrame method named `distinct` which returns only the distinct rows - see the documentation [here](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct).

Because we are humans (assuming ...), we like data to be sorted, so we will add the DataFrame `orderBy` method and sort first by state and then by city.

In [0]:
df_busSelected.select("state", "city").\
distinct().orderBy("state", "city").show(100, truncate=False)

+-----+---------------------------+
state|city |
+-----+---------------------------+
AB | |
AB |Airdrie |
AB |Alberta |
AB |Balzac |
AB |Beltline |
AB |CALGARY |
AB |Calgary |
AB |Chestermere |
AB |Delacour |
AB |Division No. 6 |
AB |Dorval |
AB |Downtown |
AB |East Calgary |
AB |Edmonton |
AB |Evergreen |
AB |Highland Park |
AB |Medicine Hat |
AB |Midnapore |
AB |North York |
AB |Northeast Calgary |
AB |Northwest Calgary |
AB |Rockey View |
AB |Rocky View |
AB |Rocky View County |
AB |Rocky View No. 44 |
AB |Rockyview |
AB |Rockyview County |
AB |SW Calgary |
AB |Sage Hill |
AB |Sainte-Adèle |
AB |Southeast Calgary |
AB |Symons Valley |
AB |calgary |
AK |Berea |
AL |Calgary |
AL |Chandler |
AL |Henderson Nevada |
AR |Avondale |
AZ | |
AZ |AZ |
AZ |Ahwahtukee |
AZ |Ahwatukee |
AZ |Ahwatukee Foothills Village|
AZ |Anthem |
AZ |Apache Junction |
AZ |Apache Trail |
AZ |Arizona |
AZ |Arrowhead |
AZ |Avondale |
AZ |Bapchule |
AZ |Buckeye |
AZ |CAVE CREEK |
AZ |CHANDLER |
AZ |Carefree |
AZ |Cave Creek |
AZ |Cave Creek Road |
AZ |Cave creek |
AZ |Central |
AZ |Central City |
AZ |Central City Village |
AZ |Chander |
AZ |Chandler |
AZ |Chandler-Gilbert |
AZ |Desert Ridge |
AZ |East Mesa |
AZ |El Mirage |
AZ |Fort McDowell |
AZ |Fort Mcdowell |
AZ |Fountain Hills |
AZ |Fountain Hls |
AZ |GILBERT |
AZ |GLENDALE |
AZ |GLendale |
AZ |Gelndale |
AZ |Gilbert |
AZ |Glbert |
AZ |Glendale |
AZ |Glendale Az |
AZ |Goodyear |
AZ |Greenway |
AZ |Guadalupe |
AZ |Higley |
AZ |Kingman |
AZ |LITCHFIELD PK |
AZ |Las Vegas |
AZ |Laveen |
AZ |Laveen Village |
AZ |Litchfield |
AZ |Litchfield Park |
AZ |Litchfield park |
AZ |MESA |
AZ |Maricopa |
AZ |Mes |
AZ |Mesa |
AZ |Mesa Arizona |
AZ |Mesa Az |
AZ |Mesa, AZ |
AZ |Metro Phoenix |
AZ |New River |
AZ |North Phoenix |
+-----+---------------------------+
only showing top 100 rows

## There are more than 10 cities

Our profiling provided valuable information - cities are not each considered a metro area.

We will take a step back - how many businesses are there in each state?  Is there one metro area per state?  Are there multiple metro areas in a state (and also in our data)?  For example, if our data included
California, both Los Angeles and San Francisco could be separate metro areas in the data.

First let's look at the states.

The DataFrame class has a method named `groupBy` that will group our data based on a field or fields we specify 
(documentation is <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy" target="_blank">here</a>).

You can think of `groupBy` as being like if you had a deck of cards and you wanted to group them by suit (hearts, clubs, etc.).  We want to 
group the business data by state.  This may seem similar to using `distinct`, but where that method got rid of duplicates, `groupBy` allows us to 
do calculations on each group.  

The difference can be explained with a deck of cards.  If you asked your younger sister, "what are the distinct suits in a deck of cards", 
you would get four values back: diamonds, spades, clubs, and hearts.  If you asked her to group the cards by suit, you could then ask her to count how many there are in 
each suit.  If all of the cards are there, the answer would be 13 for each suit.

After using the DataFrame `groupBy` method, we no longer have a DataFrame.  As stated in the documentation, 
we now have an instance of the `GroupedData` class as documented <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData" target="_blank">here</a>.

The `GroupedData` class has a number of methods we can use to calculate information about each group (e.g., the mean, average, etc.). We want to get a count of the number of 
businesses in each state, so we will use the `count` method documented <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count" target="_blank">here</a>,
which returns a DataFrame, so we can call the `show` method on it.

## Step 4: Counting businesses by state
Add the code to sort with the state containing the most businesses listed first.

In [0]:
df_busSelected.select("state").groupBy("state").count().orderBy("count",ascending=False).show(100, truncate=False)

+-----+-----+
state|count|
+-----+-----+
AZ |60803|
NV |39084|
ON |36627|
OH |16392|
NC |16218|
PA |12376|
QC |10233|
AB |8682 |
WI |5525 |
IL |2034 |
SC |1328 |
CA |23 |
NY |22 |
TX |6 |
WA |5 |
AL |3 |
GA |3 |
FL |3 |
NE |2 |
BC |2 |
VT |2 |
UT |2 |
CO |2 |
CT |2 |
MI |2 |
HPL |1 |
AK |1 |
DOW |1 |
AR |1 |
XWY |1 |
HI |1 |
MO |1 |
YT |1 |
OR |1 |
VA |1 |
DUR |1 |
MB |1 |
+-----+-----+

## Step 5: Phoenix is Rising
For the state of Arizona (state code AZ), get a count of the businesses in each *city* in that state and sort the cities in descending order.  This could tell us if there is a specific metro area in the data from Arizona.

To do this, you first need one other DataFrame method, `filter`.  Whereas `select` allowed us to choose to include only specific columns, `filter` allows us to include only specific rows that match some
criteria specified in our filter.  The following cell filters our business data to include only the rows for those businesses in Arizona.

### Your First Mission:
In the following cell we are filtering to get only the rows where the state is Arizona (AZ).

We want to count the number of businesses in each *city* in Arizona and show the count so the city with the most businesses is listed first.

In [0]:
df_busSelected.filter("state = 'AZ'").show()




+--------------------+--------------------+----------+-----+------------+-----+--------------------+----------------------+
 business_id| name| city|state|review_count|stars| hours|RestaurantsPriceRange2|
+--------------------+--------------------+----------+-----+------------+-----+--------------------+----------------------+
Yzvjg0SayhoZgCljU...| Carlos Santo, NMD|Scottsdale| AZ| 4| 5.0| null| null|
51M2Kk903DFYI6gnB...|USE MY GUY SERVIC...| Mesa| AZ| 26| 4.5|[9:0-16:0, 0:0-0:...| null|
cKyLV5oWZJ2NudWgq...|Oasis Auto Center...| Gilbert| AZ| 38| 4.5|[7:0-18:0, 7:0-18...| null|
ScYkbYNkDgCneBrD9...|Junction Tire & A...| Mesa| AZ| 18| 5.0|[7:30-17:0, 7:30-...| null|
JjJs3o60uQCfctDjs...| Convertabath| Chandler| AZ| 10| 2.5|[9:0-16:0, 9:0-16...| null|
nIEhsGbw0vJuYl05b...| Bloom & Blueprint| Tempe| AZ| 7| 4.5| null| null|
Vwo64kNYDjKi98gUU...|Gateway Fit Body ...| Mesa| AZ| 16| 4.5|[5:30-20:0, 5:30-...| null|
x3Po6tJGb729u_HJP...| Glendale Glitters| Glendale| AZ| 52| 2.5|[18:0-22:0,, 18:0...| 2|
07cgbTbANYhVDfzTM...| Frontier Automotive| Phoenix| AZ| 17| 4.5|[9:0-17:0, 7:30-1...| null|
DCsS3SgVFO56F6wRO...|Missy Donuts & Co...| Mesa| AZ| 7| 2.5|[0:0-0:0, 0:0-0:0...| 1|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Phoenix| AZ| 10| 4.0|[10:0-21:0, 10:0-...| 1|
UqBTL1dq9QcOISikg...| GK's Vapor Pub| Surprise| AZ| 8| 5.0|[10:0-20:0, 10:0-...| 2|
kHCTmEekJJwYsJEy7...|Banner - Universi...| Phoenix| AZ| 3| 3.0|[8:0-17:0, 8:0-17...| null|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Tempe| AZ| 9| 4.0|[7:0-20:0, 7:0-20...| 1|
xFc50drSPxXkcLvX5...|Boomer's Sweet Ho...| Glendale| AZ| 12| 3.5| null| 1|
jutfk7U4GV899q6qf...|Christ's Church o...| Anthem| AZ| 4| 5.0|[, 8:30-16:30, 17...| null|
Sd75ucXKoZUM2BEfB...| China Gourmet| Mesa| AZ| 13| 3.0| null| 1|
Mmd5WDFq9hHcQ3uCl...| Irene's Tap Room| Phoenix| AZ| 79| 4.5|[12:0-2:0, 0:0-0:...| null|
gflx3mAzi4iG20NOU...|BeStitched Needle...|Scottsdale| AZ| 11| 3.5|[10:0-15:0,, 10:0...| null|
YFsb1ydMxFLrxtJ3C...| LA Fitness| Phoenix| AZ| 32| 2.5|[5:0-21:0, 8:0-16...| null|
+--------------------+--------------------+----------+-----+------------+-----+--------------------+----------------------+
only showing top 20 rows

In [0]:
df_busSelected.filter("state = 'AZ'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show()

+---------------+-----+
 city|count|
+---------------+-----+
 Phoenix|20170|
 Scottsdale| 9341|
 Mesa| 6577|
 Tempe| 4797|
 Chandler| 4579|
 Glendale| 3822|
 Gilbert| 3743|
 Peoria| 2080|
 Surprise| 1217|
 Goodyear| 951|
 Avondale| 743|
 Cave Creek| 424|
 Fountain Hills| 369|
 Sun City| 300|
Litchfield Park| 251|
 Tolleson| 168|
Paradise Valley| 141|
 El Mirage| 135|
 Laveen| 135|
 Sun City West| 114|
+---------------+-----+
only showing top 20 rows

In [0]:
df_busSelected.filter("state = 'AZ'",).groupBy("state").count().orderBy("count",ascending=False).show()

+-----+-----+
state|count|
+-----+-----+
 AZ|60803|
+-----+-----+

# Your Mission: Find the 10 Metro Areas

Find the top city for each of the top 11 states (it drops off fast after that).

Use markdown to record the state / metro area combinations (complete the table):

| State | Metro Area | Size |
| ------- | ------------ |
| AZ  |  Phoenix  |  60803 |
| NV  |  Las Vegas  |  39084 |
| ON  |  Toronto  |  36627 |
|OH   | Cleveland |  16392|
|NC   |Charlotte|16218|
|PA   |Pittsburg|12376|
|QC   |Montréal| 10233|
|AB   |Calgary| 8682 |
|WI   | Madison |5525 |
|IL   |Champaign| 2034 |
|SC| Fort Mill|1328|


The cells below should include your code for determining the metro areas for each of the top 11 states.

In [0]:
df_busSelected.filter("state = 'NV'").select("city").groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'AZ'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'ON'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state =='OH'").select("city").groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'NC'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'PA'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'QC'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'AB'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'WI'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'IL'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)
df_busSelected.filter("state = 'SC'").select("city",).groupBy("city").count().orderBy("count","city", ascending=False).show(3,truncate=False)

+---------------+-----+
city |count|
+---------------+-----+
Las Vegas |31623|
Henderson |5272 |
North Las Vegas|1677 |
+---------------+-----+
only showing top 3 rows

+----------+-----+
city |count|
+----------+-----+
Phoenix |20170|
Scottsdale|9341 |
Mesa |6577 |
+----------+-----+
only showing top 3 rows

+-----------+-----+
city |count|
+-----------+-----+
Toronto |20364|
Mississauga|3518 |
Markham |1962 |
+-----------+-----+
only showing top 3 rows

+---------+-----+
city |count|
+---------+-----+
Cleveland|3856 |
Lakewood |562 |
Mentor |559 |
+---------+-----+
only showing top 3 rows

+---------+-----+
city |count|
+---------+-----+
Charlotte|10417|
Concord |974 |
Matthews |812 |
+---------+-----+
only showing top 3 rows

+-----------+-----+
city |count|
+-----------+-----+
Pittsburgh |7630 |
Monroeville|442 |
Wexford |302 |
+-----------+-----+
only showing top 3 rows

+--------+-----+
city |count|
+--------+-----+
Montréal|6979 |
Laval |472 |
Brossard|209 |
+--------+-----+
only showing top 3 rows

+-----------+-----+
city |count|
+-----------+-----+
Calgary |8375 |
Airdrie |186 |
Chestermere|37 |
+-----------+-----+
only showing top 3 rows

+-----------+-----+
city |count|
+-----------+-----+
Madison |3684 |
Middleton |392 |
Sun Prairie|276 |
+-----------+-----+
only showing top 3 rows

+---------+-----+
city |count|
+---------+-----+
Champaign|1327 |
Urbana |425 |
Savoy |83 |
+---------+-----+
only showing top 3 rows

+-----------+-----+
city |count|
+-----------+-----+
Fort Mill |693 |
Rock Hill |346 |
Indian Land|106 |
+-----------+-----+
only showing top 3 rows

In [0]:
df_busSelected.select("state").groupBy("state").count().orderBy("count",ascending=False).show(11, truncate=False)

+-----+-----+
state|count|
+-----+-----+
AZ |60803|
NV |39084|
ON |36627|
OH |16392|
NC |16218|
PA |12376|
QC |10233|
AB |8682 |
WI |5525 |
IL |2034 |
SC |1328 |
+-----+-----+
only showing top 11 rows

In [0]:
df_busSelected.filter("state = 'AZ'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'NV'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'ON'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'OH'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'NC'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'PA'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'QC'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'AB'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'WI'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'IL'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)
df_busSelected.filter("state = 'SC'").select("city",).groupBy("city").count().orderBy("count", ascending=False).show(1,truncate=False)

+-------+-----+
city |count|
+-------+-----+
Phoenix|20170|
+-------+-----+
only showing top 1 row

+---------+-----+
city |count|
+---------+-----+
Las Vegas|31623|
+---------+-----+
only showing top 1 row

+-------+-----+
city |count|
+-------+-----+
Toronto|20364|
+-------+-----+
only showing top 1 row

+---------+-----+
city |count|
+---------+-----+
Cleveland|3856 |
+---------+-----+
only showing top 1 row

+---------+-----+
city |count|
+---------+-----+
Charlotte|10417|
+---------+-----+
only showing top 1 row

+----------+-----+
city |count|
+----------+-----+
Pittsburgh|7630 |
+----------+-----+
only showing top 1 row

+--------+-----+
city |count|
+--------+-----+
Montréal|6979 |
+--------+-----+
only showing top 1 row

+-------+-----+
city |count|
+-------+-----+
Calgary|8375 |
+-------+-----+
only showing top 1 row

+-------+-----+
city |count|
+-------+-----+
Madison|3684 |
+-------+-----+
only showing top 1 row

+---------+-----+
city |count|
+---------+-----+
Champaign|1327 |
+---------+-----+
only showing top 1 row

+---------+-----+
city |count|
+---------+-----+
Fort Mill|693 |
+---------+-----+
only showing top 1 row

In [0]:
df_busSelected.select("city").groupBy("city").count().orderBy("count",ascending=False).show(10, truncate=False)

+----------+-----+
city |count|
+----------+-----+
Las Vegas |31631|
Toronto |20366|
Phoenix |20171|
Charlotte |10422|
Scottsdale|9342 |
Calgary |8377 |
Pittsburgh|7630 |
Montréal |6979 |
Mesa |6577 |
Henderson |5272 |
+----------+-----+
only showing top 10 rows

# Deliverable

* After completing your mission for the top 11 states, publish your notebook and upload the URL as the deliverable for this assignment.
* * It is recommended that you **test** the URL you submit.  To do this, first log out of Databricks and then put your URL in the browser.  Did you see what you expected? (If you are still logged into Databricks, the wrong URL can also work.)

#### We will come back to the question of identifying metro areas in the data later in the semester when we are working in Tableau.